In [2]:
from typing import Dict, TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END
import json
from pydantic import BaseModel

# Define our state
class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    current_location: Dict | None

# Define structured output for coordinates
class LocationCoordinates(BaseModel):
    latitude: float
    longitude: float
    confidence: float
    description: str

# Create tools for the agent
tools = [
    {
        "name": "analyze_image",
        "description": "Analyzes an image to identify distinguishing features, landmarks, and geographical indicators",
        "parameters": {
            "type": "object",
            "properties": {
                "image_description": {
                    "type": "string",
                    "description": "Description of the image to analyze"
                }
            },
            "required": ["image_description"]
        }
    }
]

# Initialize the LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Create the React agent
react_agent = create_react_agent(
    llm=llm,
    tools=tools,
    system_message="""You are an expert at analyzing images and determining their geographical location.
    Use the analyze_image tool to process images and identify key features that can help determine the location.
    Your goal is to predict the most likely location where the photo was taken."""
)

# Function to process agent output into structured coordinates
def process_location_to_coordinates(state: AgentState) -> AgentState:
    # Extract location information from agent's messages
    messages = state["messages"]
    last_message = messages[-1].content
    
    # Parse the location prediction and convert to coordinates
    # This is a simplified example - in practice, you might use a geocoding service
    try:
        # Example structured output
        coordinates = LocationCoordinates(
            latitude=40.7128,  # Example coordinates for New York
            longitude=-74.0060,
            confidence=0.85,
            description="Based on the architectural style and urban environment..."
        )
        
        state["current_location"] = coordinates.model_dump()
    except Exception as e:
        state["current_location"] = {"error": str(e)}
    
    return state

# Create the graph
workflow = StateGraph(AgentState)

# Add the React agent node
workflow.add_node("agent", react_agent)

# Add the coordinates processing node
workflow.add_node("process_coordinates", process_location_to_coordinates)

# Add edges
workflow.add_edge("agent", "process_coordinates")
workflow.add_edge("process_coordinates", END)

# Set entry point
workflow.set_entry_point("agent")

# Compile the graph
app = workflow.compile()

# Example usage
def predict_location(image_description: str):
    result = app.invoke(
        {
            "messages": [
                HumanMessage(content=f"Please analyze this image and determine its location: {image_description}")
            ],
            "current_location": None
        }
    )
    return result["current_location"]

# Example usage
if __name__ == "__main__":
    # Example image description
    image_description = """A busy street with tall skyscrapers, yellow taxis, and 
    people walking on wide sidewalks. There's a large electronic billboard visible 
    and what appears to be Times Square signage."""
    
    result = predict_location(image_description)
    print(json.dumps(result, indent=2))

TypeError: create_react_agent() got an unexpected keyword argument 'llm'